## Re_test Single Model

In [76]:
%run Functions.py
import warnings
warnings.filterwarnings('ignore')

In [60]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')
macro = pd.read_csv('macro_c.csv')

In [61]:
train,test = get_additional_features2(train,test)

## Price SQM as Target

In [62]:
train['life_sq_ratio'] = train['life_sq']/train['full_sq']
train['kitch_sq_ratio'] = train['kitch_sq']/train['full_sq']
test['life_sq_ratio'] = test['life_sq']/test['full_sq']
test['kitch_sq_ratio'] = test['kitch_sq']/test['full_sq']

In [44]:
Price_sqm = [x/y for x,y in zip(list(train['price_doc']*0.95+10),list(train['full_sq']))]

In [45]:
col = list(test.columns)[2:]
col = [each for each in col if each not in ['full_sq','life_sq','kitch_sq']]
train['price_sqm'] = Price_sqm
label = train['price_sqm']

In [21]:
dtrain = xgb.DMatrix(train[col],train['price_sqm'])

In [13]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)
print(len(xgb_cvalid))

[0]	train-rmse:133513+614.837	test-rmse:133514+1251.8
[50]	train-rmse:34749.9+362.496	test-rmse:37396.1+1828.55
[100]	train-rmse:30665.1+342.071	test-rmse:35201.6+1791.8
[150]	train-rmse:29086+269.026	test-rmse:34881.4+1834.53
[200]	train-rmse:27856.7+264.026	test-rmse:34727.2+1822.09
[250]	train-rmse:26773+252.794	test-rmse:34665.8+1827.22
[300]	train-rmse:25856.5+275.036	test-rmse:34597.8+1823.6
[350]	train-rmse:25017.1+275.499	test-rmse:34568.2+1875.36
[400]	train-rmse:24257.9+281.502	test-rmse:34547.2+1871.69
400


In [26]:
Model1_feature,Model1_test,indexs = get_xgb_stack_data(params,400,train,col,label,test)

TRAIN: [ 5853  5854  5855 ..., 29259 29260 29261] TEST: [   0    1    2 ..., 5850 5851 5852]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [ 5853  5854  5855 ..., 11703 11704 11705]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [11706 11707 11708 ..., 17555 17556 17557]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [17558 17559 17560 ..., 23407 23408 23409]
TRAIN: [    0     1     2 ..., 23407 23408 23409] TEST: [23410 23411 23412 ..., 29259 29260 29261]
Start Training


In [136]:
bakup = Model1_test

In [137]:
Model1_test = Model1_test*test['full_sq']

In [29]:
indexs = indexs[1:]

In [33]:
from sklearn.metrics import mean_squared_error as MSE
accu = []
for each in indexs:
    features = Model1_feature.iloc[each,:]
    predicted = features['predicted']*features['full_sq']
    label = features['label']*features['full_sq']
    accuracy=MSE(np.log(predicted+1),np.log(label+1))**0.5
    accu.append(accuracy)
    print(accuracy)

0.322588065986
0.337926213036
0.322001773804
0.298429765688


In [34]:
print(np.mean(accu))

0.320236454628


## Price SQM as Target --Remve first 5000

In [73]:
train = pd.read_csv('train_clean.csv')
train = train.iloc[5000:,:]
test = pd.read_csv('test_clean.csv')
macro = pd.read_csv('macro_c.csv')
train,test = get_additional_features2(train,test)
train['life_sq_ratio'] = train['life_sq']/train['full_sq']
train['kitch_sq_ratio'] = train['kitch_sq']/train['full_sq']
test['life_sq_ratio'] = test['life_sq']/test['full_sq']
test['kitch_sq_ratio'] = test['kitch_sq']/test['full_sq']

In [74]:
Price_sqm = [x/y for x,y in zip(list(train['price_doc']*0.95+10),list(train['full_sq']))]
col = list(test.columns)[2:]
col = [each for each in col if each not in ['full_sq','life_sq','kitch_sq']]
train['price_sqm'] = Price_sqm
label = train['price_sqm']
dtrain = xgb.DMatrix(train[col],train['price_sqm'])

In [75]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)
print(len(xgb_cvalid))

[0]	train-rmse:134097+248.75	test-rmse:134103+511.083
[50]	train-rmse:32946.1+311.399	test-rmse:34832.1+624.058
[100]	train-rmse:28726.5+224.466	test-rmse:32365.8+633.555
[150]	train-rmse:27127.1+201.852	test-rmse:32013.9+616.942
[200]	train-rmse:25818.4+141.12	test-rmse:31830.7+569.746
[250]	train-rmse:24764.9+178.537	test-rmse:31716.2+552.022
[300]	train-rmse:23805+178.769	test-rmse:31649.8+516.069
[350]	train-rmse:22977.5+115.397	test-rmse:31630.1+506.791
379


In [77]:
Model1_feature2,Model1_test2,indexs2 = get_xgb_stack_data(params,379,train,col,train['price_sqm'],test)

TRAIN: [ 4853  4854  4855 ..., 24259 24260 24261] TEST: [   0    1    2 ..., 4850 4851 4852]
TRAIN: [    0     1     2 ..., 24259 24260 24261] TEST: [4853 4854 4855 ..., 9703 9704 9705]
TRAIN: [    0     1     2 ..., 24259 24260 24261] TEST: [ 9706  9707  9708 ..., 14555 14556 14557]
TRAIN: [    0     1     2 ..., 24259 24260 24261] TEST: [14558 14559 14560 ..., 19407 19408 19409]
TRAIN: [    0     1     2 ..., 19407 19408 19409] TEST: [19410 19411 19412 ..., 24259 24260 24261]
Start Training


In [134]:
Model1_test2=Model1_test2*test['full_sq']

In [86]:
accu = []
for each in indexs2:
    features = Model1_feature2.iloc[each,:]
    predicted = features['predicted']*features['full_sq']
    label = features['price_sqm']*features['full_sq']
    accuracy=MSE(np.log(predicted+1),np.log(label+1))**0.5
    accu.append(accuracy)
    print(accuracy)

0.330499890798
0.33008175114
0.344211435143
0.307070079518
0.301070450625


In [87]:
np.mean(accu)

0.3225867214448751

# Price Gap Quarterly As Label

In [116]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')
macro = pd.read_csv('macro_c.csv')
quarter= pd.read_csv('Quaterly Data.csv')

In [117]:
train,test = get_additional_features2(train,test)

In [118]:
train['life_sq_ratio'] = train['life_sq']/train['full_sq']
train['kitch_sq_ratio'] = train['kitch_sq']/train['full_sq']
test['life_sq_ratio'] = test['life_sq']/test['full_sq']
test['kitch_sq_ratio'] = test['kitch_sq']/test['full_sq']

In [119]:
def quarter_cal(Month):
    if Month>=1 and Month <=3:
        return 'Q1'
    elif Month >3 and Month <=6:
        return 'Q2'
    elif Month >6 and Month <=9:
        return 'Q3'
    else:
        return 'Q4'

In [120]:
train_quarter = [str(x)+quarter_cal(y) for x,y in zip(list(train.timestamp.dt.year),list(train.month))]
test_quarter = [str(x)+quarter_cal(y) for x,y in zip(list(test.timestamp.dt.year),list(test.month))]
train['Quarter'] = train_quarter
test['Quarter'] = test_quarter
quarter_data = pd.read_csv('Quaterly Data.csv',sep=';')[['Quarter','Primary Average']]
train = train.merge(quarter_data,on='Quarter',how='left')
test = test.merge(quarter_data,on='Quarter',how='left')
Price_sqm = [x/y for x,y in zip(list(train['price_doc']*0.95+10),list(train['full_sq']))]

In [121]:
del train['Quarter']
del test['Quarter']
Price_gap = Price_sqm - train['Primary Average']
train['Gap'] = Price_gap
col = list(test.columns)[2:]
col = [each for each in col if each not in ['full_sq','life_sq','kitch_sq','Primary Average']]

In [122]:
label = train['Gap']
dtrain = xgb.DMatrix(train[col],train['Gap'])

In [123]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1,
    'lambda':5
}

In [124]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)
print(len(xgb_cvalid))

[0]	train-rmse:70304.4+766.808	test-rmse:70304.3+1536.41
[50]	train-rmse:34370.3+740.636	test-rmse:36184.6+1864.08
[100]	train-rmse:31421.2+541.575	test-rmse:34910.7+1861.02
[150]	train-rmse:29832.7+485.793	test-rmse:34572.1+1852.52
[200]	train-rmse:28640.4+432.476	test-rmse:34417.8+1848.99
[250]	train-rmse:27645.7+372.604	test-rmse:34334.6+1864.77
[300]	train-rmse:26731.8+397.347	test-rmse:34274.2+1859.95
326


In [125]:
Model2_feature,Model2_test,indexs2 = get_xgb_stack_data(params,326,train,col,label,test)

TRAIN: [ 5853  5854  5855 ..., 29259 29260 29261] TEST: [   0    1    2 ..., 5850 5851 5852]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [ 5853  5854  5855 ..., 11703 11704 11705]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [11706 11707 11708 ..., 17555 17556 17557]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [17558 17559 17560 ..., 23407 23408 23409]
TRAIN: [    0     1     2 ..., 23407 23408 23409] TEST: [23410 23411 23412 ..., 29259 29260 29261]
Start Training


In [141]:
bakup = Model2_test

In [143]:
Model2_test = (Model2_test+test['Primary Average'])*test['full_sq']

In [115]:
Model2_feature['Price_sqm'] = Price_sqm

In [126]:
indexs = indexs2[1:]
accu = []
for each in indexs:
    features = Model2_feature.iloc[each,:]
    predicted = (features['predicted']+features['Primary Average'])*features['full_sq']
    label = (features['label']+features['Primary Average'])*features['full_sq']
    accuracy=MSE(np.log(predicted+1),np.log(label+1))**0.5
    accu.append(accuracy)
    print(accuracy)

0.319675484273
0.334657222225
0.325784710517
0.296904911606


In [127]:
np.mean(accu)

0.31925558215514638

## Price Gap Quarterly- Remove First 5000

In [128]:
train = pd.read_csv('train_clean.csv')
train = train.iloc[5000:,:]
test = pd.read_csv('test_clean.csv')
macro = pd.read_csv('macro_c.csv')
quarter= pd.read_csv('Quaterly Data.csv')
train,test = get_additional_features2(train,test)
train['life_sq_ratio'] = train['life_sq']/train['full_sq']
train['kitch_sq_ratio'] = train['kitch_sq']/train['full_sq']
test['life_sq_ratio'] = test['life_sq']/test['full_sq']
test['kitch_sq_ratio'] = test['kitch_sq']/test['full_sq']
train_quarter = [str(x)+quarter_cal(y) for x,y in zip(list(train.timestamp.dt.year),list(train.month))]
test_quarter = [str(x)+quarter_cal(y) for x,y in zip(list(test.timestamp.dt.year),list(test.month))]
train['Quarter'] = train_quarter
test['Quarter'] = test_quarter
quarter_data = pd.read_csv('Quaterly Data.csv',sep=';')[['Quarter','Primary Average']]
train = train.merge(quarter_data,on='Quarter',how='left')
test = test.merge(quarter_data,on='Quarter',how='left')
Price_sqm = [x/y for x,y in zip(list(train['price_doc']*0.95+10),list(train['full_sq']))]
del train['Quarter']
del test['Quarter']
Price_gap = Price_sqm - train['Primary Average']
train['Gap'] = Price_gap
col = list(test.columns)[2:]
col = [each for each in col if each not in ['full_sq','life_sq','kitch_sq','Primary Average']]
label = train['Gap']
dtrain = xgb.DMatrix(train[col],train['Gap'])

In [129]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)
print(len(xgb_cvalid))

[0]	train-rmse:69803.8+319.757	test-rmse:69826.6+634.547
[50]	train-rmse:31870+336.974	test-rmse:33572.8+671.578
[100]	train-rmse:29029.8+296.651	test-rmse:32195.2+668.897
[150]	train-rmse:27558+309.143	test-rmse:31837.2+617.63
[200]	train-rmse:26343.4+287.524	test-rmse:31658.1+591.336
[250]	train-rmse:25345.1+291.14	test-rmse:31559.4+539.513
[300]	train-rmse:24454.4+285.749	test-rmse:31484.3+511.714
[350]	train-rmse:23606.1+260.568	test-rmse:31436.6+505.319
376


In [130]:
Model2_feature2,Model2_test2,indexs2 = get_xgb_stack_data(params,376,train,col,label,test)

TRAIN: [ 4853  4854  4855 ..., 24259 24260 24261] TEST: [   0    1    2 ..., 4850 4851 4852]
TRAIN: [    0     1     2 ..., 24259 24260 24261] TEST: [4853 4854 4855 ..., 9703 9704 9705]
TRAIN: [    0     1     2 ..., 24259 24260 24261] TEST: [ 9706  9707  9708 ..., 14555 14556 14557]
TRAIN: [    0     1     2 ..., 24259 24260 24261] TEST: [14558 14559 14560 ..., 19407 19408 19409]
TRAIN: [    0     1     2 ..., 19407 19408 19409] TEST: [19410 19411 19412 ..., 24259 24260 24261]
Start Training


In [145]:
bakup = Model2_test2

In [146]:
Model2_test2 = (Model2_test2+test['Primary Average'])*test['full_sq']

In [131]:
indexs = indexs2
accu = []
for each in indexs:
    features = Model2_feature2.iloc[each,:]
    predicted = (features['predicted']+features['Primary Average'])*features['full_sq']
    label = (features['label']+features['Primary Average'])*features['full_sq']
    accuracy=MSE(np.log(predicted+1),np.log(label+1))**0.5
    accu.append(accuracy)
    print(accuracy)

0.325166678408
0.323444295119
0.344661626878
0.307476193444
0.295349659968


In [132]:
np.mean(accu)

0.31921969076365164

## Tide Up Results

In [183]:
train_stacking = pd.DataFrame({'Price_SQM':Model1_feature['predicted']*Model1_feature['full_sq'],'Price_Gap':(Model2_feature['predicted']+Model2_feature['Primary Average'])*Model2_feature['full_sq']})

In [184]:
train_stacking['label'] = Model1_feature['price_doc']
train_stacking['id'] = Model1_feature['id']

In [185]:
train_stacking.to_csv('Stacking_Input_SQMGap_Complete.csv',index=False)

In [186]:
train_stacking['timestamp'] = Model1_feature['timestamp']

In [187]:
train_stacking2 = pd.DataFrame({'Price_SQM':Model1_feature2['predicted']*Model1_feature2['full_sq'],'Price_Gap':(Model2_feature2['predicted']+Model2_feature2['Primary Average'])*Model2_feature2['full_sq']})

In [188]:
train_stacking2['label'] = Model1_feature2['price_doc']
train_stacking2['id'] = Model1_feature2['id']
train_stacking2['timestamp'] = Model1_feature2['timestamp']

In [189]:
train_stacking2.to_csv('Stacking_Input_SQMGap_non5000.csv',index=False)

In [190]:
test_stacking = pd.DataFrame({'Price_SQM':Model1_test,'Price_Gap':Model2_test})

In [191]:
test_stacking['id'] = test['id']
test_stacking['timestamp'] = test['timestamp']

In [192]:
test_stacking2 = pd.DataFrame({'Price_SQM':Model1_test2,'Price_Gap':Model2_test2})

In [193]:
test_stacking2['id'] = test['id']
test_stacking2['timestamp'] = test['timestamp']

In [194]:
test_stacking.to_csv('Stacking_Predict_SQMGap_Complete.csv',index=False)
test_stacking2.to_csv('Stacking_Predict_SQMGap_non5000.csv',index=False)